In [ ]:
import os
import csv
import time
import numpy as np
from sonnmf.main import sonnmf
from sonnmf.old.main import sonnmf as sonnmf_old
import matplotlib.pyplot as plt

In [ ]:
def save_results(filepath, W, H, fscores, gscores, hscores, total_scores):
    with open(filepath, 'wb') as fout:
        np.savez_compressed(fout, W=W, H=H, fscores=fscores, gscores=gscores, hscores=hscores, total_scores=total_scores)

def load_results(filepath):
    data = np.load(filepath)
    return data['W'], data['H'], data['fscores'], data['gscores'], data['hscores'], data['total_scores']

def save_results_old(filepath, W, H, fscores, gscores, total_scores):
    with open(filepath, 'wb') as fout:
        np.savez_compressed(fout, W=W, H=H, fscores=fscores, gscores=gscores, total_scores=total_scores)

def load_results_old(filepath):
    data = np.load(filepath)
    return data['W'], data['H'], data['fscores'], data['gscores'], data['total_scores']

In [ ]:
csv_filename = "../saved_models/runtime_comparison/results.csv"

if not os.path.exists(csv_filename):
    with open(csv_filename, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['Dataset' ,'Method', 'Time Taken', 'Save Filepath'])

    #  with open(csv_filename, 'a', newline='') as csv_file:
    #                 csv_writer = csv.writer(csv_file)
    #                 csv_writer.writerow(['Dataset' ,'Method', 'Time Taken', 'Save Filepath'])

In [ ]:
w_solvers = ['proximal_avg_with_penalty_func', 'proximal_avg_with_indicator_func', 'admm', 'nesterov_smoothing', 'subgradient']

## synthetic (r=8)

In [ ]:
data_filepath = '../datasets/synthetic_data.npz'
ini_filepath = '../saved_models/synthetic/r{}_ini.npz'

In [ ]:
if os.path.exists(data_filepath):
    data = np.load(data_filepath)
    M = data['M']
    W_true = data['W_true']
    H_true = data['H_true']


In [ ]:
dataset_name = 'sythetic_r8'
m, n = M.shape
r = 8

max_iters = 1000
_lam = 1e-6
_gamma = 1.5

In [ ]:
if os.path.exists(ini_filepath.format(r)):
    data = np.load(ini_filepath.format(r))
    ini_W = data['ini_W']
    ini_H = data['ini_H']
else:
    ini_W = np.random.rand(m, r)
    ini_H = np.random.rand(r, n)
    with open(ini_filepath.format(r), 'wb') as fout:
        np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

In [ ]:
# Proximal averaging with penalty term

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'

if not os.path.exists(save_filepath):
    start_time = time.time()
    W, H, fscores, gscores, hscores, total_scores = sonnmf(M, ini_W.copy(), ini_H.copy(), lam=_lam, gamma=_gamma, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
    time_taken = time.time() - start_time
    save_results(save_filepath, W, H, fscores, gscores, hscores, total_scores)

    with open(csv_filename, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([dataset_name , w_solvers[0], time_taken, save_filepath])

In [ ]:
for w_solver in w_solvers[1:]:
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'

    if not os.path.exists(save_filepath):
        start_time = time.time()
        W, H, fscores, gscores, total_scores = sonnmf_old(M, ini_W.copy(), ini_H.copy(), lam=_lam, w_update_method=w_solver, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
        time_taken = time.time() - start_time
        save_results_old(save_filepath, W, H, fscores, gscores, total_scores)

        with open(csv_filename, 'a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([dataset_name , w_solver, time_taken, save_filepath])

In [ ]:
w_solvers = ['proximal_avg_with_penalty_func', 'proximal_avg_with_indicator_func', 'admm', 'nesterov_smoothing']
linech = ['-', '--', '-.', ':']

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'
_, _, fscores, gscores, _, total_scores = load_results(save_filepath)
total_scores = fscores + _lam * gscores
plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[0])

for i in range(1, len(w_solvers)):
    w_solver = w_solvers[i]
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'
    _, _, fscores, gscores, total_scores = load_results_old(save_filepath)
    # plt.plot(total_scores)
    plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[i])



plt.yscale('log')
plt.legend(['Proximal Avg. w/ penalty', 'Proximal Avg. w/ indicator', 'ADMM', 'Nesterov Smoothing'])
plt.xlabel('Iterations')
plt.ylabel(r'$F(W, H) = \frac{1}{2} \| WH - M \|_F^2 + \lambda \sum_{i \neq j} \| w_i - w_j \|_2$')
plt.tight_layout()
plt.savefig(f'F_clipped_{dataset_name}.jpg')


## jasper region 1 (8 * 8)

In [ ]:
data_filepath = '../datasets/jasper_region_1.npz'
ini_filepath = '../saved_models/jasper_region_1/r{}_ini.npz'

In [ ]:
M = np.load(data_filepath)['X']
M = M.astype(np.float64)
m, n = M.shape

In [ ]:
dataset_name = 'jasper_region_1'
m, n = M.shape
r = n

max_iters = 2000
_lam = 40000
_gamma = 10000

In [ ]:
if os.path.exists(ini_filepath.format(r)):
    data = np.load(ini_filepath.format(r))
    ini_W = data['ini_W']
    ini_H = data['ini_H']
else:
    ini_W = np.random.rand(m, r)
    ini_H = np.random.rand(r, n)
    with open(ini_filepath.format(r), 'wb') as fout:
        np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

In [ ]:
# Proximal averaging with penalty term

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'

if not os.path.exists(save_filepath):
    start_time = time.time()
    W, H, fscores, gscores, hscores, total_scores = sonnmf(M, ini_W.copy(), ini_H.copy(), lam=_lam, gamma=_gamma, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
    time_taken = time.time() - start_time
    save_results(save_filepath, W, H, fscores, gscores, hscores, total_scores)

    with open(csv_filename, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([dataset_name , w_solvers[0], time_taken, save_filepath])

In [ ]:
for w_solver in w_solvers[1:]:
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'

    if not os.path.exists(save_filepath):
        start_time = time.time()
        W, H, fscores, gscores, total_scores = sonnmf_old(M, ini_W.copy(), ini_H.copy(), lam=_lam, w_update_method=w_solver, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
        time_taken = time.time() - start_time
        save_results_old(save_filepath, W, H, fscores, gscores, total_scores)

        with open(csv_filename, 'a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([dataset_name , w_solver, time_taken, save_filepath])

In [ ]:
w_solvers = ['proximal_avg_with_penalty_func', 'proximal_avg_with_indicator_func', 'admm', 'nesterov_smoothing']
linech = ['-', '--', '-.', ':']

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'
_, _, fscores, gscores, _, total_scores = load_results(save_filepath)
total_scores = fscores + _lam * gscores
plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[0])

for i in range(1, len(w_solvers)):
    w_solver = w_solvers[i]
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'
    _, _, fscores, gscores, total_scores = load_results_old(save_filepath)
    # plt.plot(total_scores)
    plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[i])



plt.yscale('log')
plt.legend(['Proximal Avg. w/ penalty', 'Proximal Avg. w/ indicator', 'ADMM', 'Nesterov Smoothing'])
plt.xlabel('Iterations')
plt.ylabel(r'$F(W, H) = \frac{1}{2} \| WH - M \|_F^2 + \lambda \sum_{i \neq j} \| w_i - w_j \|_2$')
plt.tight_layout()
plt.savefig(f'F_clipped_{dataset_name}.jpg')


## jasper full

In [ ]:
data_filepath = '../datasets/jasper_full.npz'
ini_filepath = '../saved_models/jasper_fullr{}_ini.npz'

In [ ]:
M = np.load(data_filepath)['X']
M = M.astype(np.float64)
m, n = M.shape

In [ ]:
dataset_name = 'jasper_full'
m, n = M.shape
r = n

max_iters = 1000
_lam = 1000000
_gamma = 1000000

In [ ]:
if os.path.exists(ini_filepath.format(r)):
    data = np.load(ini_filepath.format(r))
    ini_W = data['ini_W']
    ini_H = data['ini_H']
else:
    ini_W = np.random.rand(m, r)
    ini_H = np.random.rand(r, n)
    with open(ini_filepath.format(r), 'wb') as fout:
        np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

In [ ]:
# Proximal averaging with penalty term

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'

if not os.path.exists(save_filepath):
    start_time = time.time()
    W, H, fscores, gscores, hscores, total_scores = sonnmf(M, ini_W.copy(), ini_H.copy(), lam=_lam, gamma=_gamma, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
    time_taken = time.time() - start_time
    save_results(save_filepath, W, H, fscores, gscores, hscores, total_scores)

    with open(csv_filename, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([dataset_name , w_solvers[0], time_taken, save_filepath])

In [ ]:
for w_solver in w_solvers[1:]:
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'

    if not os.path.exists(save_filepath):
        start_time = time.time()
        W, H, fscores, gscores, total_scores = sonnmf_old(M, ini_W.copy(), ini_H.copy(), lam=_lam, w_update_method=w_solver, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
        time_taken = time.time() - start_time
        save_results_old(save_filepath, W, H, fscores, gscores, total_scores)

        with open(csv_filename, 'a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([dataset_name , w_solver, time_taken, save_filepath])

In [ ]:
w_solvers = ['proximal_avg_with_penalty_func', 'proximal_avg_with_indicator_func', 'admm', 'nesterov_smoothing']
linech = ['-', '--', '-.', ':']

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'
_, _, fscores, gscores, _, total_scores = load_results(save_filepath)
total_scores = fscores + _lam * gscores
plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[0])

for i in range(1, len(w_solvers)):
    w_solver = w_solvers[i]
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'
    _, _, fscores, gscores, total_scores = load_results_old(save_filepath)
    # plt.plot(total_scores)
    plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[i])



plt.yscale('log')
plt.legend(['Proximal Avg. w/ penalty', 'Proximal Avg. w/ indicator', 'ADMM', 'Nesterov Smoothing'])
plt.xlabel('Iterations')
plt.ylabel(r'$F(W, H) = \frac{1}{2} \| WH - M \|_F^2 + \lambda \sum_{i \neq j} \| w_i - w_j \|_2$')
plt.tight_layout()
plt.savefig(f'F_clipped_{dataset_name}.jpg')


## urban full

In [ ]:
data_filepath = '../datasets/urban_full.npz'
ini_filepath = '../saved_models/urban_fullr{}_ini.npz'

In [ ]:
M = np.load(data_filepath)['X']
M = M.astype(np.float64)
m, n = M.shape

In [ ]:
dataset_name = 'urban_full'
m, n = M.shape
r = n

max_iters = 1000
_lam = 1000000
_gamma = 1000000

In [ ]:
if os.path.exists(ini_filepath.format(r)):
    data = np.load(ini_filepath.format(r))
    ini_W = data['ini_W']
    ini_H = data['ini_H']
else:
    ini_W = np.random.rand(m, r)
    ini_H = np.random.rand(r, n)
    with open(ini_filepath.format(r), 'wb') as fout:
        np.savez_compressed(fout, ini_W=ini_W, ini_H=ini_H)

In [ ]:
# Proximal averaging with penalty term

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'

if not os.path.exists(save_filepath):
    start_time = time.time()
    W, H, fscores, gscores, hscores, total_scores = sonnmf(M, ini_W.copy(), ini_H.copy(), lam=_lam, gamma=_gamma, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
    time_taken = time.time() - start_time
    save_results(save_filepath, W, H, fscores, gscores, hscores, total_scores)

    with open(csv_filename, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow([dataset_name , w_solvers[0], time_taken, save_filepath])

In [ ]:
for w_solver in w_solvers[1:]:
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'

    if not os.path.exists(save_filepath):
        start_time = time.time()
        W, H, fscores, gscores, total_scores = sonnmf_old(M, ini_W.copy(), ini_H.copy(), lam=_lam, w_update_method=w_solver, itermax=max_iters, W_update_iters=10, early_stop=True, verbose=False)
        time_taken = time.time() - start_time
        save_results_old(save_filepath, W, H, fscores, gscores, total_scores)

        with open(csv_filename, 'a', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow([dataset_name , w_solver, time_taken, save_filepath])

In [ ]:
w_solvers = ['proximal_avg_with_penalty_func', 'proximal_avg_with_indicator_func', 'admm', 'nesterov_smoothing']
linech = ['-', '--', '-.', ':']

save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solvers[0]}.npz'
_, _, fscores, gscores, _, total_scores = load_results(save_filepath)
total_scores = fscores + _lam * gscores
plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[0])

for i in range(1, len(w_solvers)):
    w_solver = w_solvers[i]
    save_filepath = f'../saved_models/runtime_comparison/{dataset_name}_{w_solver}.npz'
    _, _, fscores, gscores, total_scores = load_results_old(save_filepath)
    # plt.plot(total_scores)
    plt.plot(total_scores[np.insert(np.abs(total_scores[1:] - total_scores[:-1]) / total_scores[:-1] >= 1e-6, 0, True)], linech[i])



plt.yscale('log')
plt.legend(['Proximal Avg. w/ penalty', 'Proximal Avg. w/ indicator', 'ADMM', 'Nesterov Smoothing'])
plt.xlabel('Iterations')
plt.ylabel(r'$F(W, H) = \frac{1}{2} \| WH - M \|_F^2 + \lambda \sum_{i \neq j} \| w_i - w_j \|_2$')
plt.tight_layout()
plt.savefig(f'F_clipped_{dataset_name}.jpg')
